### **實作進階優化技術**

我們將使用 CIFAR-10 資料集來展示如何應用這些技術來優化模型訓練。我們將使用 Keras 來實作權重衰減和學習率調整，並應用於一個簡單的卷積神經網路。

#### **Keras 實作：進階優化技術**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.datasets import cifar10

# 下載並預處理 CIFAR-10 資料集
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# 將數據正規化到 [0, 1] 範圍
x_train = x_train.astype('float32') / 255.0  # 將訓練數據的像素值縮放到 0 到 1 之間
x_test = x_test.astype('float32') / 255.0    # 將測試數據的像素值縮放到 0 到 1 之間

# 定義學習率調整函數（以指數衰減為例）
def lr_schedule(epoch):
    initial_lr = 0.001  # 初始學習率
    return initial_lr * tf.math.exp(-0.1 * epoch)  # 每個 epoch 學習率按指數衰減

# 構建 CNN 模型
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3), 
                  kernel_regularizer=regularizers.l2(0.001)),  # 第一個卷積層，32 個 3x3 的濾波器，ReLU 激活函數，L2 正則化
    layers.MaxPooling2D((2, 2)),  # 最大池化層，池化窗口大小為 2x2
    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),  # 第二個卷積層，64 個 3x3 的濾波器，ReLU 激活函數，L2 正則化
    layers.MaxPooling2D((2, 2)),  # 最大池化層，池化窗口大小為 2x2
    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),  # 第三個卷積層，64 個 3x3 的濾波器，ReLU 激活函數，L2 正則化
    layers.MaxPooling2D((2, 2)),  # 最大池化層，池化窗口大小為 2x2
    layers.Flatten(),  # 將多維輸出展平成一維
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),  # 全連接層，64 個神經元，ReLU 激活函數，L2 正則化
    layers.Dense(10, activation='softmax')  # 輸出層，10 個神經元（對應 10 個類別），softmax 激活函數
])

# 編譯模型
model.compile(optimizer='adam',  # 使用 Adam 優化器
              loss='sparse_categorical_crossentropy',  # 損失函數為稀疏分類交叉熵
              metrics=['accuracy'])  # 評估指標為準確率

# 訓練模型，應用學習率調整
history = model.fit(x_train, y_train, epochs=20, batch_size=64,  # 訓練 20 個 epoch，每批次 64 個樣本
                    validation_data=(x_test, y_test),  # 使用測試數據進行驗證
                    callbacks=[LearningRateScheduler(lr_schedule)])  # 使用學習率調整回調函數


### **使用 PyTorch 實作進階優化技術**

同樣的，我們也可以使用 PyTorch 來實現這些進階優化技術。

#### **PyTorch 實作：進階優化技術**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 資料增強與預處理
transform = transforms.Compose([
    transforms.ToTensor(),  # 將圖像轉換為張量
    transforms.Normalize((0.5,), (0.5,))  # 正規化圖像數據，使其均值為 0.5，標準差為 0.5
])

# CIFAR-10 資料集
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)  # 下載並加載訓練數據集
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)  # 將訓練數據集加載到 DataLoader 中，批次大小為 64，並打亂數據

test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)  # 下載並加載測試數據集
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)  # 將測試數據集加載到 DataLoader 中，批次大小為 64，不打亂數據

# 建立 CNN 模型
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)  # 第一個卷積層，輸入通道數為 3，輸出通道數為 32，卷積核大小為 3x3，padding 為 1
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)  # 第二個卷積層，輸出通道數為 64
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)  # 第三個卷積層，輸出通道數為 64
        self.pool = nn.MaxPool2d(2, 2)  # 最大池化層，池化窗口大小為 2x2
        self.fc1 = nn.Linear(64 * 4 * 4, 64)  # 全連接層，輸入大小為 64*4*4，輸出大小為 64
        self.fc2 = nn.Linear(64, 10)  # 輸出層，輸出大小為 10（對應 10 個類別）

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))  # 第一個卷積層後接 ReLU 激活函數和最大池化層
        x = self.pool(torch.relu(self.conv2(x)))  # 第二個卷積層後接 ReLU 激活函數和最大池化層
        x = self.pool(torch.relu(self.conv3(x)))  # 第三個卷積層後接 ReLU 激活函數和最大池化層
        x = x.view(-1, 64 * 4 * 4)  # 將多維輸出展平成一維
        x = torch.relu(self.fc1(x))  # 全連接層後接 ReLU 激活函數
        x = self.fc2(x)  # 輸出層
        return x

# 初始化模型
model = CNN()

# 定義損失函數與優化器（加入權重衰減）
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)  # 使用 Adam 優化器，學習率為 0.001，加入權重衰減
criterion = nn.CrossEntropyLoss()  # 使用交叉熵損失函數

# 定義學習率調整策略（使用指數衰減）
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)  # 每個 epoch 學習率按 0.9 的比例衰減

# 訓練模型
for epoch in range(20):  # 訓練 20 個 epoch
    model.train()  # 設置模型為訓練模式
    running_loss = 0.0
    for images, labels in train_loader:  # 遍歷訓練數據
        optimizer.zero_grad()  # 清空梯度
        outputs = model(images)  # 前向傳播
        loss = criterion(outputs, labels)  # 計算損失
        loss.backward()  # 反向傳播
        optimizer.step()  # 更新參數
    
    # 調整學習率
    scheduler.step()
    
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')  # 輸出每個 epoch 的損失值